In [21]:
#import modules
from keras.layers import *
from keras.utils import load_img, img_to_array
import pandas as pd 
import numpy as np 
import os
from keras.models import load_model
from tqdm import tqdm

In [22]:
image_height = 200
image_weight = 150
sequence = 3
sequence_length = 20

def arrange_data(df, map):
    image_data = []
    ids = df[df.columns[0]]
    map = f"{map}/raw_images/"
    
    df = df.drop(df.columns[0], axis=1)
    
    for i in ids:
        img = load_img(f"{map}{i}.jpg", target_size=(image_height, image_weight, sequence))
        img = img_to_array(img)
        image_data.append(img)
    
    X = np.array(image_data)
    Y = np.array(df)
    
    print("Shape of images:", X.shape)
    print("Shape of labels:", Y.shape)
    
    return X, Y

In [23]:
# splitting data and scale
current_folder = os.path.abspath('')
map = f"{current_folder}/data/"

# reads first row double
print("Processing test..")
val_df = pd.read_csv(f"{map}test.csv", delimiter=',', header=0)
X_test, Y_test = arrange_data(val_df, current_folder)

# scale data 
X_test = X_test / 255
Y_test = Y_test / 255


Processing test..
Shape of images: (25, 200, 150, 3)
Shape of labels: (25, 26)


In [27]:
def accuracy_score(map, model_path):
    
    test_df = pd.read_csv(f"{map}test.csv", delimiter=',', header=None)
    X_test, Y_test = arrange_data(test_df, current_folder)

    model = load_model(model_path)

    pred = model.predict(np.array(X_test))

    count = 0
    for i in tqdm(range(len(pred))):
        value = 0
        
        first3_index = np.argsort(pred[i])[-3:]
        correct = np.where(Y_test[i] == 1)[0]
        
        for j in first3_index:
            if j in correct:
                value += 1
                
        if (value>0):
            count=count+1
                
    print("Images having atleast one genre correctly identified=",count)
    print("Total number of images =",len(pred))
    print("Accuracy = ", count/len(pred))

In [31]:

accuracy_score(map, f"{current_folder}/result_model/generated.h5")
print("\n")
accuracy_score(map, f"{current_folder}/result_model/pretrained.h5")

Shape of images: (26, 200, 150, 3)
Shape of labels: (26, 26)
1/1 [==============================] - 0s 179ms/step


100%|██████████| 26/26 [00:00<00:00, 26020.50it/s]

Images having atleast one genre correctly identified= 19
Total number of images = 26
Accuracy =  0.7307692307692307


Shape of images: (26, 200, 150, 3)
Shape of labels: (26, 26)


1/1 [==============================] - 2s 2s/step


100%|██████████| 26/26 [00:00<00:00, 26008.09it/s]

Images having atleast one genre correctly identified= 19
Total number of images = 26
Accuracy =  0.7307692307692307
